## 1. Setup and Data Loading

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

In [ ]:
# Load original data with LoanApproved for threshold analysis
df = pd.read_csv('../data/data_raw/Loan_data.csv')

print(f"Data loaded: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nLoanApproved distribution:")
print(df['LoanApproved'].value_counts())

In [ ]:
# Load model comparison results
results_df = pd.read_csv('../models/model_comparison_results.csv')
best_model_name = results_df.loc[results_df['Test RMSE'].idxmin(), 'Model']

print(f"Best Model: {best_model_name}")
print(f"Test RMSE: {results_df['Test RMSE'].min():.4f}")
print(f"Test R²: {results_df.loc[results_df['Test RMSE'].idxmin(), 'Test R2']:.4f}")

In [ ]:
# Load the best model and generate predictions on full dataset
linear_models = ['Ridge Regression', 'Lasso Regression', 'ElasticNet']
catboost_models = ['CatBoost']

if best_model_name in linear_models:
    dataset_path = '../data/data_processed/dataset_linear_models.pkl'
    model_file = best_model_name.lower().replace(' ', '_')
elif best_model_name in catboost_models:
    dataset_path = '../data/data_processed/dataset_catboost.pkl'
    model_file = 'catboost'
else:
    dataset_path = '../data/data_processed/dataset_tree_models.pkl'
    model_file = best_model_name.lower().replace(' ', '_')

# Load dataset
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)
    X_train = data['X_train']
    X_test = data['X_test']
    y_train = data['y_train']
    y_test = data['y_test']

# Load model
model_path = f'../models/{model_file}_best.pkl'
with open(model_path, 'rb') as f:
    model = pickle.load(f)

# Generate predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f"\nPredictions generated:")
print(f"  Training predictions: {y_train_pred.shape}")
print(f"  Test predictions: {y_test_pred.shape}")

## 2. Historical Analysis: RiskScore vs LoanApproved

Analyzing the relationship between RiskScore and historical loan approval decisions.

In [ ]:
# Convert LoanApproved to binary if needed
if df['LoanApproved'].dtype == 'object':
    df['LoanApproved_Binary'] = (df['LoanApproved'] == 'Yes').astype(int)
else:
    df['LoanApproved_Binary'] = df['LoanApproved']

# Analyze RiskScore distribution by approval status
approved_stats = df[df['LoanApproved_Binary'] == 1]['RiskScore'].describe()
rejected_stats = df[df['LoanApproved_Binary'] == 0]['RiskScore'].describe()

print("="*80)
print("HISTORICAL RISKSCORE ANALYSIS")
print("="*80)
print("\nAPPROVED Loans:")
print(approved_stats)
print("\nREJECTED Loans:")
print(rejected_stats)
print("\nKey Observations:")
print(f"  • Approved loans average RiskScore: {approved_stats['mean']:.2f}")
print(f"  • Rejected loans average RiskScore: {rejected_stats['mean']:.2f}")
print(f"  • Difference: {rejected_stats['mean'] - approved_stats['mean']:.2f} points")

In [ ]:
# Visualize RiskScore distributions
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution comparison
axes[0].hist(df[df['LoanApproved_Binary'] == 1]['RiskScore'], bins=50, alpha=0.6, 
             label='Approved', color='green', edgecolor='black')
axes[0].hist(df[df['LoanApproved_Binary'] == 0]['RiskScore'], bins=50, alpha=0.6, 
             label='Rejected', color='red', edgecolor='black')
axes[0].axvline(approved_stats['mean'], color='green', linestyle='--', linewidth=2, label=f'Approved Mean: {approved_stats["mean"]:.1f}')
axes[0].axvline(rejected_stats['mean'], color='red', linestyle='--', linewidth=2, label=f'Rejected Mean: {rejected_stats["mean"]:.1f}')
axes[0].set_xlabel('RiskScore', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('RiskScore Distribution: Approved vs Rejected Loans', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
df_plot = df[['RiskScore', 'LoanApproved']].copy()
sns.boxplot(data=df_plot, x='LoanApproved', y='RiskScore', ax=axes[1], palette=['red', 'green'])
axes[1].set_xlabel('Loan Approval Status', fontsize=12)
axes[1].set_ylabel('RiskScore', fontsize=12)
axes[1].set_title('RiskScore by Approval Status', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate overlap zone
approved_risk = df[df['LoanApproved_Binary'] == 1]['RiskScore']
rejected_risk = df[df['LoanApproved_Binary'] == 0]['RiskScore']
overlap_min = max(approved_risk.min(), rejected_risk.min())
overlap_max = min(approved_risk.max(), rejected_risk.max())

print(f"\nOVERLAP ZONE IDENTIFIED:")
print(f"  Range: {overlap_min:.2f} - {overlap_max:.2f}")
print(f"  Width: {overlap_max - overlap_min:.2f} points")
print(f"\n  → This is the 'gray zone' where manual review should occur")

In [ ]:
# Calculate approval rate by RiskScore bins
df['RiskScore_Bin'] = pd.cut(df['RiskScore'], bins=20)
approval_by_bin = df.groupby('RiskScore_Bin', observed=True).agg({
    'LoanApproved_Binary': ['mean', 'count']
}).round(3)
approval_by_bin.columns = ['Approval_Rate', 'Count']
approval_by_bin['RiskScore_Mid'] = approval_by_bin.index.map(lambda x: x.mid)

# Plot approval rate curve
plt.figure(figsize=(14, 6))
plt.plot(approval_by_bin['RiskScore_Mid'], approval_by_bin['Approval_Rate'], 
         marker='o', linewidth=2, markersize=8, color='steelblue')
plt.axhline(0.5, color='orange', linestyle='--', linewidth=2, label='50% Approval Rate')
plt.xlabel('RiskScore', fontsize=12)
plt.ylabel('Historical Approval Rate', fontsize=12)
plt.title('Historical Loan Approval Rate by RiskScore', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Find the threshold where approval rate = 50%
threshold_50 = approval_by_bin.loc[(approval_by_bin['Approval_Rate'] - 0.5).abs().idxmin(), 'RiskScore_Mid']
print(f"\nHistorical 50% approval threshold: RiskScore ≈ {threshold_50:.2f}")

## 3. Threshold Optimization: Cost-Benefit Analysis

Defining optimal thresholds based on business costs.

In [ ]:
# Define business parameters (adjustable)
print("="*80)
print("BUSINESS PARAMETERS FOR THRESHOLD OPTIMIZATION")
print("="*80)

# Average loan amount (from data)
avg_loan_amount = df['LoanAmount'].mean()
print(f"\nAverage Loan Amount: €{avg_loan_amount:,.2f}")

# Business costs (hypothetical - adjust based on bank's data)
INTEREST_RATE = 0.05  # 5% annual interest
AVG_LOAN_DURATION_YEARS = df['LoanDuration'].mean() / 12  # Convert months to years
PROFIT_PER_GOOD_LOAN = avg_loan_amount * INTEREST_RATE * AVG_LOAN_DURATION_YEARS
LOSS_PER_DEFAULT = avg_loan_amount * 0.70  # Assume 70% loss on default (recovery ~30%)
COST_MANUAL_REVIEW = 50  # Cost per manual review

print(f"\nBusiness Economics:")
print(f"  • Interest Rate: {INTEREST_RATE*100}%")
print(f"  • Average Loan Duration: {AVG_LOAN_DURATION_YEARS:.2f} years")
print(f"  • Profit per GOOD loan: €{PROFIT_PER_GOOD_LOAN:,.2f}")
print(f"  • Loss per DEFAULT: €{LOSS_PER_DEFAULT:,.2f}")
print(f"  • Cost per manual review: €{COST_MANUAL_REVIEW:,.2f}")
print(f"\nProfit/Loss Ratio: 1:{LOSS_PER_DEFAULT/PROFIT_PER_GOOD_LOAN:.2f}")
print(f"  → One default erases profit from {LOSS_PER_DEFAULT/PROFIT_PER_GOOD_LOAN:.1f} good loans!")

In [ ]:
# Create analysis dataset with predictions
# Combine train and test for full analysis
full_predictions = pd.DataFrame({
    'Actual_RiskScore': pd.concat([y_train, y_test]),
    'Predicted_RiskScore': np.concatenate([y_train_pred, y_test_pred])
})

# For threshold analysis, we'll use predicted RiskScore
# Assume relationship: Higher RiskScore → Higher default probability
# Simplified model: P(default) = RiskScore / 100
full_predictions['Default_Probability'] = full_predictions['Predicted_RiskScore'] / 100

print(f"Analysis dataset created: {full_predictions.shape}")
print(f"\nPredicted RiskScore statistics:")
print(full_predictions['Predicted_RiskScore'].describe())

In [ ]:
def calculate_expected_value(threshold_low, threshold_high, predictions_df, 
                              profit_good, loss_default, cost_review):
    """
    Calculate expected value for a 3-zone threshold strategy.
    
    Zones:
    - GREEN (0 to threshold_low): Auto-approve
    - ORANGE (threshold_low to threshold_high): Manual review
    - RED (threshold_high to 100): Auto-reject
    """
    df = predictions_df.copy()
    
    # Classify applications
    df['Zone'] = pd.cut(df['Predicted_RiskScore'], 
                        bins=[0, threshold_low, threshold_high, 100],
                        labels=['GREEN', 'ORANGE', 'RED'],
                        include_lowest=True)
    
    # Calculate outcomes for each zone
    green_count = (df['Zone'] == 'GREEN').sum()
    orange_count = (df['Zone'] == 'ORANGE').sum()
    red_count = (df['Zone'] == 'RED').sum()
    
    # GREEN zone: Auto-approve all
    green_df = df[df['Zone'] == 'GREEN']
    green_expected_profit = green_df.apply(
        lambda x: (1 - x['Default_Probability']) * profit_good - x['Default_Probability'] * loss_default,
        axis=1
    ).sum()
    
    # ORANGE zone: Manual review (assume approve 70% after review, reject risky 30%)
    orange_df = df[df['Zone'] == 'ORANGE']
    orange_review_cost = orange_count * cost_review
    # Assume manual review correctly identifies 80% of high-risk cases
    orange_approval_rate = 0.70
    orange_approved = orange_df.sample(frac=orange_approval_rate, random_state=42) if len(orange_df) > 0 else orange_df
    orange_expected_profit = orange_approved.apply(
        lambda x: (1 - x['Default_Probability']) * profit_good - x['Default_Probability'] * loss_default,
        axis=1
    ).sum() - orange_review_cost
    
    # RED zone: Auto-reject (no profit, no loss)
    red_expected_profit = 0
    
    # Total
    total_expected_profit = green_expected_profit + orange_expected_profit + red_expected_profit
    total_applications = len(df)
    total_approved = green_count + int(orange_count * orange_approval_rate)
    approval_rate = total_approved / total_applications
    
    return {
        'threshold_low': threshold_low,
        'threshold_high': threshold_high,
        'total_profit': total_expected_profit,
        'profit_per_application': total_expected_profit / total_applications,
        'approval_rate': approval_rate,
        'green_count': green_count,
        'orange_count': orange_count,
        'red_count': red_count,
        'green_pct': green_count / total_applications * 100,
        'orange_pct': orange_count / total_applications * 100,
        'red_pct': red_count / total_applications * 100
    }

print("Threshold evaluation function defined.")

In [ ]:
# Test different threshold strategies
print("="*80)
print("EVALUATING THRESHOLD STRATEGIES")
print("="*80)

strategies = [
    {'name': 'Conservative', 'low': 30, 'high': 55},
    {'name': 'Balanced', 'low': 35, 'high': 65},
    {'name': 'Aggressive', 'low': 40, 'high': 70},
    {'name': 'Very Aggressive', 'low': 45, 'high': 75},
]

results = []
for strategy in strategies:
    result = calculate_expected_value(
        strategy['low'], strategy['high'], full_predictions,
        PROFIT_PER_GOOD_LOAN, LOSS_PER_DEFAULT, COST_MANUAL_REVIEW
    )
    result['strategy'] = strategy['name']
    results.append(result)
    
    print(f"\n{strategy['name'].upper()} Strategy:")
    print(f"  Thresholds: [{strategy['low']}, {strategy['high']}]")
    print(f"  Total Expected Profit: €{result['total_profit']:,.2f}")
    print(f"  Profit per Application: €{result['profit_per_application']:.2f}")
    print(f"  Approval Rate: {result['approval_rate']*100:.1f}%")
    print(f"  Distribution: GREEN {result['green_pct']:.1f}% | ORANGE {result['orange_pct']:.1f}% | RED {result['red_pct']:.1f}%")

results_df_thresholds = pd.DataFrame(results)

In [ ]:
# Visualize strategy comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total profit
axes[0, 0].bar(results_df_thresholds['strategy'], results_df_thresholds['total_profit'], color='steelblue')
axes[0, 0].set_ylabel('Total Expected Profit (€)', fontsize=11)
axes[0, 0].set_title('Total Expected Profit by Strategy', fontsize=12, fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Profit per application
axes[0, 1].bar(results_df_thresholds['strategy'], results_df_thresholds['profit_per_application'], color='seagreen')
axes[0, 1].set_ylabel('Profit per Application (€)', fontsize=11)
axes[0, 1].set_title('Profit per Application by Strategy', fontsize=12, fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Approval rate
axes[1, 0].bar(results_df_thresholds['strategy'], results_df_thresholds['approval_rate']*100, color='coral')
axes[1, 0].set_ylabel('Approval Rate (%)', fontsize=11)
axes[1, 0].set_title('Approval Rate by Strategy', fontsize=12, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Zone distribution (stacked bar)
x = range(len(results_df_thresholds))
axes[1, 1].bar(x, results_df_thresholds['green_pct'], label='GREEN (Auto-OK)', color='green', alpha=0.7)
axes[1, 1].bar(x, results_df_thresholds['orange_pct'], bottom=results_df_thresholds['green_pct'], 
               label='ORANGE (Review)', color='orange', alpha=0.7)
axes[1, 1].bar(x, results_df_thresholds['red_pct'], 
               bottom=results_df_thresholds['green_pct']+results_df_thresholds['orange_pct'], 
               label='RED (Auto-Reject)', color='red', alpha=0.7)
axes[1, 1].set_ylabel('Percentage of Applications (%)', fontsize=11)
axes[1, 1].set_title('Application Distribution by Zone', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(results_df_thresholds['strategy'], rotation=45)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identify best strategy
best_strategy = results_df_thresholds.loc[results_df_thresholds['profit_per_application'].idxmax()]
print("\n" + "="*80)
print("RECOMMENDED STRATEGY")
print("="*80)
print(f"\n🏆 {best_strategy['strategy']} Strategy")
print(f"  • Thresholds: GREEN [0-{best_strategy['threshold_low']}], ORANGE [{best_strategy['threshold_low']}-{best_strategy['threshold_high']}], RED [{best_strategy['threshold_high']}-100]")
print(f"  • Expected Profit per Application: €{best_strategy['profit_per_application']:.2f}")
print(f"  • Approval Rate: {best_strategy['approval_rate']*100:.1f}%")
print(f"  • Manual Review Load: {best_strategy['orange_pct']:.1f}% of applications")

## 4. ROI Analysis: ML Model vs. Old Rule-Based System

In [ ]:
# Compare ML system to old rule-based system
print("="*80)
print("ROI ANALYSIS: ML MODEL VS. OLD SYSTEM")
print("="*80)

# Assumptions about old system
OLD_SYSTEM_APPROVAL_RATE = df['LoanApproved_Binary'].mean()
OLD_SYSTEM_AVG_RISK = df[df['LoanApproved_Binary'] == 1]['RiskScore'].mean()
OLD_SYSTEM_DEFAULT_RATE = OLD_SYSTEM_AVG_RISK / 100  # Simplified assumption

# New ML system (using best strategy)
NEW_SYSTEM_APPROVAL_RATE = best_strategy['approval_rate']
# Calculate average risk of approved applications in new system
green_mask = full_predictions['Predicted_RiskScore'] <= best_strategy['threshold_low']
orange_mask = (full_predictions['Predicted_RiskScore'] > best_strategy['threshold_low']) & \
              (full_predictions['Predicted_RiskScore'] <= best_strategy['threshold_high'])
# Assume 70% of orange zone gets approved
approved_risk_scores = pd.concat([
    full_predictions[green_mask]['Predicted_RiskScore'],
    full_predictions[orange_mask]['Predicted_RiskScore'].sample(frac=0.7, random_state=42)
])
NEW_SYSTEM_AVG_RISK = approved_risk_scores.mean()
NEW_SYSTEM_DEFAULT_RATE = NEW_SYSTEM_AVG_RISK / 100

# Annual volume (hypothetical)
ANNUAL_APPLICATIONS = 10000

# Old system
old_approvals = ANNUAL_APPLICATIONS * OLD_SYSTEM_APPROVAL_RATE
old_defaults = old_approvals * OLD_SYSTEM_DEFAULT_RATE
old_good_loans = old_approvals - old_defaults
old_profit = old_good_loans * PROFIT_PER_GOOD_LOAN - old_defaults * LOSS_PER_DEFAULT

# New system
new_approvals = ANNUAL_APPLICATIONS * NEW_SYSTEM_APPROVAL_RATE
new_defaults = new_approvals * NEW_SYSTEM_DEFAULT_RATE
new_good_loans = new_approvals - new_defaults
new_manual_reviews = ANNUAL_APPLICATIONS * (best_strategy['orange_pct'] / 100)
new_review_costs = new_manual_reviews * COST_MANUAL_REVIEW
new_profit = new_good_loans * PROFIT_PER_GOOD_LOAN - new_defaults * LOSS_PER_DEFAULT - new_review_costs

# ML development/maintenance costs (one-time + annual)
ML_DEVELOPMENT_COST = 100000  # One-time
ML_ANNUAL_MAINTENANCE = 20000  # Annual

# Calculate ROI
annual_profit_increase = new_profit - old_profit
net_first_year = annual_profit_increase - ML_DEVELOPMENT_COST - ML_ANNUAL_MAINTENANCE
net_subsequent_years = annual_profit_increase - ML_ANNUAL_MAINTENANCE
payback_period = ML_DEVELOPMENT_COST / annual_profit_increase if annual_profit_increase > 0 else float('inf')

print(f"\nAnnual Application Volume: {ANNUAL_APPLICATIONS:,}")
print(f"\n{'OLD RULE-BASED SYSTEM':^40} | {'NEW ML SYSTEM':^40}")
print("="*83)
print(f"{'Approval Rate:':30} {OLD_SYSTEM_APPROVAL_RATE*100:>8.1f}% | {NEW_SYSTEM_APPROVAL_RATE*100:>8.1f}%")
print(f"{'Approved Applications:':30} {old_approvals:>8,.0f} | {new_approvals:>8,.0f}")
print(f"{'Average Risk of Approved:':30} {OLD_SYSTEM_AVG_RISK:>8.2f} | {NEW_SYSTEM_AVG_RISK:>8.2f}")
print(f"{'Expected Defaults:':30} {old_defaults:>8,.0f} | {new_defaults:>8,.0f}")
print(f"{'Default Rate:':30} {OLD_SYSTEM_DEFAULT_RATE*100:>8.1f}% | {NEW_SYSTEM_DEFAULT_RATE*100:>8.1f}%")
print(f"{'Manual Reviews:':30} {'N/A':>8} | {new_manual_reviews:>8,.0f}")
print(f"{'Review Costs:':30} {'€0':>8} | €{new_review_costs:>7,.0f}")
print("="*83)
print(f"{'Annual Profit:':30} €{old_profit:>7,.0f} | €{new_profit:>7,.0f}")
print(f"\n{'IMPROVEMENT':^83}")
print("="*83)
print(f"Annual Profit Increase: €{annual_profit_increase:,.2f} ({(annual_profit_increase/old_profit*100):.1f}% improvement)")
print(f"Reduction in Defaults: {old_defaults - new_defaults:,.0f} fewer defaults per year")
print(f"\n{'ML SYSTEM COSTS & ROI':^83}")
print("="*83)
print(f"Development Cost (one-time): €{ML_DEVELOPMENT_COST:,.2f}")
print(f"Annual Maintenance Cost: €{ML_ANNUAL_MAINTENANCE:,.2f}")
print(f"\nNet Profit Year 1: €{net_first_year:,.2f}")
print(f"Net Profit Year 2+: €{net_subsequent_years:,.2f}")
print(f"\nPayback Period: {payback_period:.2f} years")
print(f"3-Year ROI: {((net_first_year + 2*net_subsequent_years) / ML_DEVELOPMENT_COST * 100):.1f}%")

In [ ]:
# Visualize ROI
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Profit comparison
systems = ['Old System', 'New ML System']
profits = [old_profit, new_profit]
colors = ['lightcoral', 'lightgreen']
axes[0].bar(systems, profits, color=colors, edgecolor='black', linewidth=2)
axes[0].set_ylabel('Annual Profit (€)', fontsize=12)
axes[0].set_title('Annual Profit Comparison', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(profits):
    axes[0].text(i, v + max(profits)*0.02, f'€{v:,.0f}', ha='center', fontsize=11, fontweight='bold')

# Cumulative profit over 5 years
years = range(1, 6)
old_cumulative = [old_profit * year for year in years]
new_cumulative = [
    net_first_year if year == 1 else net_first_year + net_subsequent_years * (year - 1)
    for year in years
]
axes[1].plot(years, old_cumulative, marker='o', linewidth=2, markersize=8, label='Old System', color='red')
axes[1].plot(years, new_cumulative, marker='s', linewidth=2, markersize=8, label='ML System (net)', color='green')
axes[1].set_xlabel('Years', fontsize=12)
axes[1].set_ylabel('Cumulative Net Profit (€)', fontsize=12)
axes[1].set_title('5-Year Cumulative Profit Projection', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Deployment Architecture & Implementation Plan

### 5.1 Technical Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                   LOAN APPLICATION SYSTEM                       │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│              DATA PREPROCESSING SERVICE                         │
│  • Feature extraction                                           │
│  • Data validation                                              │
│  • Encoding & scaling (using saved transformers)                │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│              ML PREDICTION SERVICE (API)                        │
│  • Load trained model                                           │
│  • Generate RiskScore prediction                                │
│  • Generate SHAP explanation                                    │
│  • Response time: <100ms                                        │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│              DECISION ENGINE                                    │
│  • Apply threshold rules                                        │
│  • Route to appropriate workflow:                               │
│    - GREEN → Auto-approve                                       │
│    - ORANGE → Manual review queue                               │
│    - RED → Auto-reject                                          │
└─────────────────────────────────────────────────────────────────┘
                              │
        ┌─────────────────────┼─────────────────────┐
        ▼                     ▼                     ▼
┌──────────────┐  ┌──────────────────┐  ┌──────────────┐
│ AUTO-APPROVE │  │ MANUAL REVIEW    │  │ AUTO-REJECT  │
│              │  │ (with SHAP       │  │              │
│              │  │  explanation)    │  │              │
└──────────────┘  └──────────────────┘  └──────────────┘
```

### 5.2 Technology Stack Recommendations

**Model Serving:**
- FastAPI or Flask for REST API
- Docker containerization
- Load balancing for scalability

**Model Storage:**
- MLflow for model versioning
- Model registry for A/B testing

**Monitoring:**
- Prometheus + Grafana for metrics
- ELK stack for logging
- Custom dashboard for business KPIs

## 6. Monitoring & Maintenance Strategy

### 6.1 Key Performance Indicators (KPIs) to Track

**Model Performance:**
1. **Prediction Accuracy:**
   - RMSE on production data (compare predicted vs actual RiskScore when outcome known)
   - MAE, R² metrics
   - Track weekly/monthly

2. **Calibration:**
   - Are predicted risks matching actual default rates?
   - Example: Applications with RiskScore 60-70 should have ~65% actual default rate

3. **Prediction Distribution:**
   - Monitor for shifts in RiskScore distribution
   - Alert if distribution changes significantly

**Business Metrics:**
1. **Approval Rate:**
   - Target: Maintain within acceptable range
   - Track by customer segment

2. **Actual Default Rate:**
   - Compare to predicted rates
   - Track by risk zone (GREEN/ORANGE/RED)

3. **Profitability:**
   - Revenue from approved loans
   - Losses from defaults
   - Net profit per application

4. **Manual Review Queue:**
   - Volume in ORANGE zone
   - Review time
   - Override rate (manual decisions differing from model)

**Operational Metrics:**
- API response time (<100ms target)
- System uptime (>99.9% target)
- Error rates

### 6.2 Model Drift Detection

**Types of Drift:**
1. **Data Drift:** Input feature distributions change
2. **Concept Drift:** Relationship between features and RiskScore changes
3. **Prediction Drift:** Model outputs change unexpectedly

**Detection Methods:**
- Statistical tests (KS test, Chi-square test) on feature distributions
- Compare current vs. training data distributions
- Alert if p-value < 0.05

**Response Plan:**
- Investigate root cause
- Retrain model if necessary
- Update thresholds if business environment changed

### 6.3 Retraining Schedule

**Regular Retraining:**
- **Quarterly:** Update model with new data
- **Annual:** Full model review and potential architecture changes

**Trigger-based Retraining:**
- Performance degradation >10%
- Significant drift detected
- Major economic changes

**A/B Testing:**
- Deploy new model to 10% of traffic
- Compare performance for 2-4 weeks
- Full rollout if successful

## 7. Risk Management & Compliance

### 7.1 Regulatory Compliance

**GDPR (Data Protection):**
- Right to explanation: Use SHAP for transparent decisions
- Data minimization: Only use necessary features
- Consent: Clear communication about automated decisions
- Data retention: Delete applicant data per policy

**Fair Lending Laws:**
- Prohibited features: No discrimination based on protected attributes
- Disparate impact analysis: Monitor approval rates by demographic groups
- Regular audits: Quarterly fairness assessments

**Model Governance:**
- Document all model changes
- Maintain audit trail of decisions
- Regular validation by independent team

### 7.2 Explainability & Transparency

**For Loan Officers:**
- SHAP explanations for every prediction
- Feature contribution breakdown
- Comparison to similar applications

**For Customers:**
- Clear rejection reasons
- Actionable guidance: "Improve your debt-to-income ratio"
- Appeal process

**For Regulators:**
- Model documentation
- Fairness reports
- Performance metrics

### 7.3 Fallback Mechanisms

**If ML System Fails:**
1. Automatic fallback to manual review
2. Alert technical team
3. Use simplified rule-based backup system

**Edge Cases:**
- Unusual applications → route to manual review
- VIP customers → manual review regardless of score
- High-value loans → additional scrutiny

## 8. Implementation Roadmap

### Phase 1: Pilot (Months 1-3)

**Objectives:**
- Deploy to one branch or product line
- 10% of applications
- Run in parallel with old system (shadow mode)

**Key Activities:**
- Build API and decision engine
- Train loan officers on SHAP explanations
- Set up monitoring dashboards
- Compare ML decisions to human decisions

**Success Criteria:**
- Agreement rate >80% with human experts
- API response time <100ms
- No system downtime >1 hour

### Phase 2: Controlled Rollout (Months 4-6)

**Objectives:**
- Expand to 50% of applications
- Auto-approve GREEN zone only
- ORANGE/RED still manual review

**Key Activities:**
- Optimize thresholds based on pilot data
- Implement A/B testing framework
- Collect feedback from loan officers
- Monitor default rates closely

**Success Criteria:**
- Default rate ≤ historical baseline
- Approval rate maintained
- Positive feedback from loan officers

### Phase 3: Full Production (Months 7-9)

**Objectives:**
- 100% of applications through ML system
- Auto-approve GREEN, auto-reject RED
- Manual review only for ORANGE zone

**Key Activities:**
- Scale infrastructure for full load
- Implement automated retraining pipeline
- Launch customer-facing explanations
- Document everything for regulators

**Success Criteria:**
- Profit improvement ≥10%
- Processing time reduction ≥50%
- Zero compliance issues

### Phase 4: Optimization (Months 10-12)

**Objectives:**
- Continuous improvement
- Advanced features (ensemble models, deep learning)
- Expand to new products

**Key Activities:**
- Quarterly model retraining
- Feature engineering based on insights
- Explore advanced techniques
- Train additional staff

**Success Criteria:**
- Sustained performance improvement
- ROI >200% by year-end

## 9. Final Recommendations Summary

In [ ]:
print("="*80)
print("EXECUTIVE SUMMARY: ML MODEL DEPLOYMENT RECOMMENDATIONS")
print("="*80)

print(f"\n1. OPTIMAL DECISION THRESHOLDS ({best_strategy['strategy']} Strategy)")
print("   " + "─"*76)
print(f"   • GREEN Zone (Auto-Approve):  RiskScore 0-{best_strategy['threshold_low']}")
print(f"   • ORANGE Zone (Manual Review): RiskScore {best_strategy['threshold_low']}-{best_strategy['threshold_high']}")
print(f"   • RED Zone (Auto-Reject):     RiskScore {best_strategy['threshold_high']}-100")
print(f"\n   Expected Outcomes:")
print(f"   - Approval Rate: {best_strategy['approval_rate']*100:.1f}%")
print(f"   - Manual Review Load: {best_strategy['orange_pct']:.1f}% of applications")
print(f"   - Profit per Application: €{best_strategy['profit_per_application']:.2f}")

print("\n2. BUSINESS IMPACT")
print("   " + "─"*76)
print(f"   • Annual Profit Increase: €{annual_profit_increase:,.0f} ({(annual_profit_increase/old_profit*100):.1f}% improvement)")
print(f"   • Reduction in Defaults: {old_defaults - new_defaults:,.0f} fewer per year")
print(f"   • Investment Payback: {payback_period:.1f} years")
print(f"   • 3-Year ROI: {((net_first_year + 2*net_subsequent_years) / ML_DEVELOPMENT_COST * 100):.0f}%")

print("\n3. IMPLEMENTATION APPROACH")
print("   " + "─"*76)
print("   • Phase 1 (Months 1-3): Pilot with 10% of applications")
print("   • Phase 2 (Months 4-6): Controlled rollout to 50%")
print("   • Phase 3 (Months 7-9): Full production deployment")
print("   • Phase 4 (Months 10-12): Optimization and expansion")

print("\n4. KEY SUCCESS FACTORS")
print("   " + "─"*76)
print("   ✓ Robust monitoring: Track prediction accuracy, default rates, profit")
print("   ✓ Regular retraining: Quarterly updates with new data")
print("   ✓ Explainability: SHAP for every decision (transparency & compliance)")
print("   ✓ Human oversight: Manual review for borderline cases (ORANGE zone)")
print("   ✓ Compliance: Fair lending practices, GDPR, audit trails")

print("\n5. RISK MITIGATION")
print("   " + "─"*76)
print("   • Fallback system if ML fails")
print("   • A/B testing before full rollout")
print("   • Regular fairness audits")
print("   • Drift detection and alerts")
print("   • Independent validation team")

print("\n6. NEXT STEPS")
print("   " + "─"*76)
print("   1. Present findings to executive leadership")
print("   2. Secure budget approval (€120K total, Year 1)")
print("   3. Assemble deployment team (data scientists, engineers, compliance)")
print("   4. Begin Phase 1 pilot in Q1")
print("   5. Establish monitoring infrastructure")
print("   6. Train loan officers on new system")

print("\n" + "="*80)
print("CONCLUSION")
print("="*80)
print("\nThe ML-based risk scoring system offers significant benefits:")
print(f"  • {(annual_profit_increase/old_profit*100):.0f}% profit improvement")
print(f"  • {((old_defaults - new_defaults)/old_defaults*100):.0f}% reduction in defaults")
print("  • Faster, more consistent decisions")
print("  • Better customer experience")
print("  • Regulatory compliance through explainability")
print("\nWith careful implementation and monitoring, this system will transform")
print("the bank's loan approval process while maintaining risk management standards.")
print("="*80)

## 10. Export Results

In [ ]:
# Save threshold analysis results
results_df_thresholds.to_csv('../models/threshold_strategy_comparison.csv', index=False)
print("✓ Threshold strategies saved to: ../models/threshold_strategy_comparison.csv")

# Save recommended thresholds
recommendation = {
    'strategy': best_strategy['strategy'],
    'green_threshold': best_strategy['threshold_low'],
    'red_threshold': best_strategy['threshold_high'],
    'expected_profit_per_app': best_strategy['profit_per_application'],
    'approval_rate': best_strategy['approval_rate'],
    'manual_review_pct': best_strategy['orange_pct']
}

import json
with open('../models/recommended_thresholds.json', 'w') as f:
    json.dump(recommendation, f, indent=2)
print("✓ Recommended thresholds saved to: ../models/recommended_thresholds.json")

print("\n" + "="*80)
print("DEPLOYMENT ANALYSIS COMPLETE!")
print("="*80)
print("\nAll recommendations and analyses have been documented.")
print("Ready for presentation to stakeholders and deployment planning.")